# Route 55,  2:30-6pm PM peak (outbound), Monday-Friday, January - July

    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',

In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import shapely.geometry
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import numpy as np
import re


#shape = shapely.geometry.asShape(shaefile_record['geometry'])

In [2]:
with open('WeGo.json') as fi:
    credentials = json.load(fi)

In [3]:
credentials
Authorization = credentials['Authorization']


In [4]:
def concat_frames(geo_response, response):
    #This function pulls in two API requests, puts one in a geodataframe the other in a dataframe and concats desired columns
    geo_df = gpd.GeoDataFrame.from_features(geo_response.json()['features'])
    res_df = pd.DataFrame(response.json()['data']['segments'])
    full_df = pd.concat([geo_df, res_df[['countForSegment', 'percent20Secs']]], axis = 1)
    return full_df


# Jan-Feb 01-25-2022 -> 02-24-2022


In [5]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/55'

params = {
    'direction': '0',
    'startDate': '01-25-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '02-24-2022',
    'endTime' : '18:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}


params1 = {
    'direction': '0',
    'startDate': '01-25-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '02-24-2022',
    'endTime' : '18:00',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

route_55_zoe1 =  concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))
route_55_zoe1 = route_55_zoe1.loc[route_55_zoe1['pathLengthMeters'] > 0]
route_55_zoe1 = route_55_zoe1.loc[route_55_zoe1['countForSegment'] > route_55_zoe1['countForSegment'].max()/2]
route_55_zoe1 = route_55_zoe1.loc[route_55_zoe1['percent20Secs'] >= route_55_zoe1.percent20Secs.quantile(0.80)]
route_55_zoe1 = route_55_zoe1.loc[route_55_zoe1['fromStopId'] != 'ARTS']

In [6]:
route_55_zoe1.percent20Secs.quantile(0.8)

3.1812000000000005

In [7]:
route_55_zoe1.sort_values('percent20Secs', ascending = False).head()

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
18,"LINESTRING (-86.77609 36.15927, -86.77595 36.1...",MXOMCCTR,Convention Center Station Outbound,NXOPBODY,Peabody Station Outbound,24.588,51.955662,0.473250,222.584,91.911,78.233,71.992,53.6350,27.562,5.670,2.153,453,13.427
546,"LINESTRING (-86.72324 36.13437, -86.72304 36.1...",MXOPLUSP,Plus Park Station Outbound,MURWILEN,Murfreesboro Pike & Wilhagan Rd Eb,24.369,23.232324,1.048926,353.110,43.510,32.307,25.073,17.7380,11.729,4.061,1.399,441,7.875
528,"LINESTRING (-86.71317 36.12819, -86.71297 36.1...",MXOTHOMP,Thompson Station Outbound,MURTHOSF,Murfreesboro Pike & Thompson Lane S,24.324,43.002569,0.565641,496.887,129.154,49.683,31.652,17.7225,11.029,3.920,1.548,432,6.618
160,"LINESTRING (-86.65453 36.05282, -86.65454 36.0...",MTVCURNN,Mt View Rd & Curtis Hollow Rd Nb,HICHICNN,Hickory Hollow Pl & Hickory Hollow,24.719,61.192090,0.403957,1283.239,71.675,18.949,13.949,9.8450,7.375,4.985,2.732,267,6.030
215,"LINESTRING (-86.77918 36.16400, -86.77905 36.1...",MCC4_15,Central 4th Ave - Bay 15,4AVCHUSN,4th Ave N & Church St Sb,24.772,24.212172,1.023122,126.542,62.719,47.743,29.468,13.0540,7.368,4.266,1.379,447,5.603


In [8]:
route_55_zoe1.sort_values('percent20Secs', ascending = False).head(1)

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
18,"LINESTRING (-86.77609 36.15927, -86.77595 36.1...",MXOMCCTR,Convention Center Station Outbound,NXOPBODY,Peabody Station Outbound,24.588,51.955662,0.47325,222.584,91.911,78.233,71.992,53.635,27.562,5.67,2.153,453,13.427


# Feb - March 02-25-2022 -> 03-26-2022

In [9]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/55'

params = {
    'direction': '0',
    'startDate': '02-25-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '03-26-2022',
    'endTime' : '18:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}


params1 = {
    'direction': '0',
    'startDate': '02-25-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '03-26-2022',
    'endTime' : '18:00',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

route_55_zoe2 =  concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))
route_55_zoe2 = route_55_zoe2.loc[route_55_zoe2['pathLengthMeters'] > 0]
route_55_zoe2 = route_55_zoe2.loc[route_55_zoe2['countForSegment'] > route_55_zoe2['countForSegment'].max()/2]
route_55_zoe2 = route_55_zoe2.loc[route_55_zoe2['percent20Secs'] >= route_55_zoe2.percent20Secs.quantile(0.80)]
route_55_zoe2 = route_55_zoe2.loc[route_55_zoe2['fromStopId'] != 'ARTS']

In [10]:
route_55_zoe2.sort_values('percent20Secs', ascending = False).head()

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
19,"LINESTRING (-86.77609 36.15927, -86.77595 36.1...",MXOMCCTR,Convention Center Station Outbound,NXOPBODY,Peabody Station Outbound,24.588,50.752293,0.484471,331.757,93.034,79.722,67.475,46.5200,18.919,4.626,1.832,331,10.022
558,"LINESTRING (-86.72324 36.13437, -86.72304 36.1...",MXOPLUSP,Plus Park Station Outbound,MURWILEN,Murfreesboro Pike & Wilhagan Rd Eb,24.369,25.031211,0.973545,254.634,46.190,34.154,29.984,19.0240,13.359,3.696,1.029,323,8.254
231,"LINESTRING (-86.77623 36.15947, -86.77609 36.1...",4AVCHUSN,4th Ave N & Church St Sb,MXOMCCTR,Convention Center Station Outbound,24.790,37.843305,0.655070,237.869,95.594,69.627,50.268,18.1705,8.130,4.034,1.847,328,6.037
227,"LINESTRING (-86.77918 36.16400, -86.77905 36.1...",MCC4_15,Central 4th Ave - Bay 15,4AVCHUSN,4th Ave N & Church St Sb,24.772,26.844578,0.922793,141.955,67.161,53.998,33.716,14.5555,8.268,4.036,0.870,322,6.000
457,"LINESTRING (-86.76835 36.15080, -86.76810 36.1...",MXOFULTN,Fulton Station Outbound,LAF1AEM,Lafayette St & 1st Ave S Eb,24.405,23.813790,1.024826,56.943,47.467,41.535,36.299,21.1050,9.020,3.686,0.352,329,5.883


In [11]:
route_55_zoe2.sort_values('percent20Secs', ascending = False).head(1)

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
19,"LINESTRING (-86.77609 36.15927, -86.77595 36.1...",MXOMCCTR,Convention Center Station Outbound,NXOPBODY,Peabody Station Outbound,24.588,50.752293,0.484471,331.757,93.034,79.722,67.475,46.52,18.919,4.626,1.832,331,10.022


# March - April 03-27-2022 -> 04-26-2022

In [12]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/55'

params = {
    'direction': '0',
    'startDate': '03-27-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '04-26-2022',
    'endTime' : '18:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}


params1 = {
    'direction': '0',
    'startDate': '03-27-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '04-26-2022',
    'endTime' : '18:00',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

route_55_zoe3 =  concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))
route_55_zoe3 = route_55_zoe3.loc[route_55_zoe3['pathLengthMeters'] > 0]
route_55_zoe3 = route_55_zoe3.loc[route_55_zoe3['countForSegment'] > route_55_zoe3['countForSegment'].max()/2]
route_55_zoe3 = route_55_zoe3.loc[route_55_zoe3['percent20Secs'] >= route_55_zoe3.percent20Secs.quantile(0.80)]
route_55_zoe3 = route_55_zoe3.loc[route_55_zoe3['fromStopId'] != 'ARTS']

In [13]:
route_55_zoe3.sort_values('percent20Secs', ascending = False).head()

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
544,"LINESTRING (-86.72324 36.13437, -86.72304 36.1...",MXOPLUSP,Plus Park Station Outbound,MURWILEN,Murfreesboro Pike & Wilhagan Rd Eb,24.369,24.771235,0.983762,310.319,43.585,34.983,30.527,20.504,15.035,6.164,0.351,409,10.887
18,"LINESTRING (-86.77609 36.15927, -86.77595 36.1...",MXOMCCTR,Convention Center Station Outbound,NXOPBODY,Peabody Station Outbound,24.588,45.421646,0.541328,337.694,94.505,78.670,65.605,33.826,12.400,4.457,0.869,418,7.850
161,"LINESTRING (-86.65453 36.05282, -86.65454 36.0...",MTVCURNN,Mt View Rd & Curtis Hollow Rd Nb,HICHICNN,Hickory Hollow Pl & Hickory Hollow,24.719,82.207841,0.300689,1509.809,250.482,27.257,17.163,10.016,7.474,5.047,3.195,227,6.380
214,"LINESTRING (-86.77918 36.16400, -86.77905 36.1...",MCC4_15,Central 4th Ave - Bay 15,4AVCHUSN,4th Ave N & Church St Sb,24.772,28.670164,0.864034,454.972,68.550,56.932,39.554,14.291,7.269,4.016,0.575,403,5.775
218,"LINESTRING (-86.77623 36.15947, -86.77609 36.1...",4AVCHUSN,4th Ave N & Church St Sb,MXOMCCTR,Convention Center Station Outbound,24.790,38.861440,0.637907,277.621,94.524,71.038,50.572,18.940,8.196,3.983,0.876,416,5.373


In [14]:
route_55_zoe3.sort_values('percent20Secs', ascending = False).head(1)

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
544,"LINESTRING (-86.72324 36.13437, -86.72304 36.1...",MXOPLUSP,Plus Park Station Outbound,MURWILEN,Murfreesboro Pike & Wilhagan Rd Eb,24.369,24.771235,0.983762,310.319,43.585,34.983,30.527,20.504,15.035,6.164,0.351,409,10.887


# April - May 04-27-2022 -> 05-25-2022

In [15]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/55'

params = {
    'direction': '0',
    'startDate': '04-27-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '18:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}


params1 = {
    'direction': '0',
    'startDate': '04-27-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '18:00',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

route_55_zoe4 =  concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))
route_55_zoe4 = route_55_zoe4.loc[route_55_zoe4['pathLengthMeters'] > 0]
route_55_zoe4 = route_55_zoe4.loc[route_55_zoe4['countForSegment'] > route_55_zoe4['countForSegment'].max()/2]
route_55_zoe4 = route_55_zoe4.loc[route_55_zoe4['percent20Secs'] >= route_55_zoe4.percent20Secs.quantile(0.80)]
route_55_zoe4 = route_55_zoe4.loc[route_55_zoe4['fromStopId'] != 'ARTS']

In [16]:
route_55_zoe4.sort_values('percent20Secs', ascending = False).head()

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
544,"LINESTRING (-86.72324 36.13437, -86.72304 36.1...",MXOPLUSP,Plus Park Station Outbound,MURWILEN,Murfreesboro Pike & Wilhagan Rd Eb,24.369,26.020882,0.936517,334.629,47.941,36.418,29.165,20.470,13.493,5.739,0.225,425,9.939
18,"LINESTRING (-86.77609 36.15927, -86.77595 36.1...",MXOMCCTR,Convention Center Station Outbound,NXOPBODY,Peabody Station Outbound,24.588,39.162907,0.627839,240.461,90.040,72.245,56.416,26.821,10.271,4.475,2.236,431,7.308
218,"LINESTRING (-86.77623 36.15947, -86.77609 36.1...",4AVCHUSN,4th Ave N & Church St Sb,MXOMCCTR,Convention Center Station Outbound,24.790,46.595700,0.532023,316.883,98.915,77.819,66.234,35.807,11.983,4.483,2.260,427,6.866
443,"LINESTRING (-86.76835 36.15080, -86.76810 36.1...",MXOFULTN,Fulton Station Outbound,LAF1AEM,Lafayette St & 1st Ave S Eb,24.405,25.146660,0.970507,66.575,48.076,42.843,36.871,24.754,9.792,3.693,1.332,427,6.050
161,"LINESTRING (-86.65453 36.05282, -86.65454 36.0...",MTVCURNN,Mt View Rd & Curtis Hollow Rd Nb,HICHICNN,Hickory Hollow Pl & Hickory Hollow,24.719,35.979411,0.687032,993.839,20.074,14.407,12.129,8.652,7.286,5.173,3.325,253,6.008


In [17]:
route_55_zoe4.sort_values('percent20Secs', ascending = False).head(1)

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
544,"LINESTRING (-86.72324 36.13437, -86.72304 36.1...",MXOPLUSP,Plus Park Station Outbound,MURWILEN,Murfreesboro Pike & Wilhagan Rd Eb,24.369,26.020882,0.936517,334.629,47.941,36.418,29.165,20.47,13.493,5.739,0.225,425,9.939


# May - June 05-26-2022 -> 06-25-2022

In [18]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/55'

params = {
    'direction': '0',
    'startDate': '05-26-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '06-25-2022',
    'endTime' : '18:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}


params1 = {
    'direction': '0',
    'startDate': '05-26-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '06-25-2022',
    'endTime' : '18:00',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

route_55_zoe5 =  concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))
route_55_zoe5 = route_55_zoe5.loc[route_55_zoe5['pathLengthMeters'] > 0]
route_55_zoe5 = route_55_zoe5.loc[route_55_zoe5['countForSegment'] > route_55_zoe5['countForSegment'].max()/2]
route_55_zoe5 = route_55_zoe5.loc[route_55_zoe5['percent20Secs'] >= route_55_zoe5.percent20Secs.quantile(0.80)]
route_55_zoe5 = route_55_zoe5.loc[route_55_zoe5['fromStopId'] != 'ARTS']

In [19]:
route_55_zoe5.sort_values('percent20Secs', ascending = False).head()

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
544,"LINESTRING (-86.72324 36.13437, -86.72304 36.1...",MXOPLUSP,Plus Park Station Outbound,MURWILEN,Murfreesboro Pike & Wilhagan Rd Eb,24.369,29.489857,0.826352,311.970,56.341,37.476,32.773,23.522,17.189,6.704,1.095,427,12.666
443,"LINESTRING (-86.76835 36.15080, -86.76810 36.1...",MXOFULTN,Fulton Station Outbound,LAF1AEM,Lafayette St & 1st Ave S Eb,24.405,27.396185,0.890817,61.619,47.267,42.455,37.732,32.048,14.602,4.303,2.010,427,8.326
18,"LINESTRING (-86.77609 36.15927, -86.77595 36.1...",MXOMCCTR,Convention Center Station Outbound,NXOPBODY,Peabody Station Outbound,24.588,44.939366,0.547137,282.935,91.912,77.688,65.857,37.113,11.288,4.209,2.104,429,7.301
214,"LINESTRING (-86.77918 36.16400, -86.77905 36.1...",MCC4_15,Central 4th Ave - Bay 15,4AVCHUSN,4th Ave N & Church St Sb,24.772,26.098943,0.949157,149.188,64.118,52.998,32.947,14.872,8.629,4.481,2.245,418,6.380
218,"LINESTRING (-86.77623 36.15947, -86.77609 36.1...",4AVCHUSN,4th Ave N & Church St Sb,MXOMCCTR,Convention Center Station Outbound,24.790,40.812720,0.607409,229.671,93.221,73.882,60.929,29.203,9.432,4.182,2.193,429,6.183


In [20]:
route_55_zoe5.sort_values('percent20Secs', ascending = False).head(1)

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
544,"LINESTRING (-86.72324 36.13437, -86.72304 36.1...",MXOPLUSP,Plus Park Station Outbound,MURWILEN,Murfreesboro Pike & Wilhagan Rd Eb,24.369,29.489857,0.826352,311.97,56.341,37.476,32.773,23.522,17.189,6.704,1.095,427,12.666


# June - July 06-26-2022 -> -7-26-2022

In [21]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/55'

params = {
    'direction': '0',
    'startDate': '06-26-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '07-26-2022',
    'endTime' : '18:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}


params1 = {
    'direction': '0',
    'startDate': '06-26-2022',
    'beginTime' : '14:30',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '07-26-2022',
    'endTime' : '18:00',
    'resolution': 'hiRes',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

route_55_zoe6 =  concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))
route_55_zoe6 = route_55_zoe6.loc[route_55_zoe6['pathLengthMeters'] > 0]
route_55_zoe6 = route_55_zoe6.loc[route_55_zoe6['countForSegment'] > route_55_zoe6['countForSegment'].max()/2]
route_55_zoe6 = route_55_zoe6.loc[route_55_zoe6['percent20Secs'] >= route_55_zoe6.percent20Secs.quantile(0.80)]
route_55_zoe6 = route_55_zoe6.loc[route_55_zoe6['fromStopId'] != 'ARTS']

In [ ]:
route_55_zoe6.sort_values('percent20Secs', ascending = False).head()

In [ ]:
route_55_zoe6.sort_values('percent20Secs', ascending = False).head(1)

In [ ]:
route_55_zoe6.value_counts('toStopName')

In [ ]:
route_55_zoe5.value_counts('toStopName')

In [ ]:
route_55_zoe4.value_counts('toStopName')

In [ ]:
route_55_zoe3.value_counts('toStopName')

In [ ]:
route_55_zoe2.value_counts('toStopName')

In [ ]:
route_55_zoe1.value_counts('toStopName')

In [ ]:
big_list = route_55_zoe6.value_counts('toStopName') + route_55_zoe5.value_counts('toStopName') + route_55_zoe4.value_counts('toStopName') + route_55_zoe3.value_counts('toStopName') + route_55_zoe2.value_counts('toStopName')

In [ ]:
big_list

In [ ]:
big_df = pd.DataFrame(big_list)

In [ ]:
big_df.sort_values(0, ascending = False).head()

# compare top

In [ ]:
top_5a = route_55_zoe1.sort_values('percent20Secs', ascending = False).head(5)
top_5a = top_5a[['fromStopName', 'toStopName']]
top_5a

In [ ]:
top_5b = route_55_zoe2.sort_values('percent20Secs', ascending = False).head(5)
top_5b = top_5b[['fromStopName', 'toStopName']]
top_5b

In [ ]:
top_5c = route_55_zoe3.sort_values('percent20Secs', ascending = False).head(5)
top_5c = top_5c[['fromStopName', 'toStopName']]
top_5c

In [ ]:
top_5c = route_55_zoe4.sort_values('percent20Secs', ascending = False).head(5)
top_5c = top_5c[['fromStopName', 'toStopName']]
top_5c

In [ ]:
top_5d = route_55_zoe5.sort_values('percent20Secs', ascending = False).head(5)
top_5d = top_5d[['fromStopName', 'toStopName']]
top_5d

In [ ]:
top_5e = route_55_zoe6.sort_values('percent20Secs', ascending = False).head(5)
top_5e = top_5e[['fromStopName', 'toStopName']]
top_5e

In [ ]:
route_55_zoe1.plot()

In [ ]:
route_55_zoe2.plot()

In [ ]:
route_55_zoe3.plot()

In [ ]:
route_55_zoe1 = route_55_zoe1['geometry'].to_json()

In [ ]:
mapdawg = folium.Map(location = [36.19, -86.7775], zoom_start = 12)
folium.GeoJson(route_55_zoe1).add_to(mapdawg)
mapdawg


In [ ]:
#pm_peak_out = pd.concat([route_55_zoe1, route_55_zoe2, route_55_zoe3, route_55_zoe4, route_55_zoe5, route_55_zoe6])
#pm_peak_out = pm_peak_out.reset_index()

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/55'

params = {
    'direction': '1',
    'startDate': '04-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'resolution': 'hiRes',  
}


params1 = {
    'direction': '1',
    'startDate': '04-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '09:00',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

m_f_am_peak_aprmay =  concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))
m_f_am_peak_aprmay = m_f_am_peak_aprmay.loc[m_f_am_peak_aprmay['pathLengthMeters'] > 0]
m_f_am_peak_aprmay = m_f_am_peak_aprmay.loc[m_f_am_peak_aprmay['countForSegment'] > m_f_am_peak_aprmay['countForSegment'].max()/2]
m_f_am_peak_aprmay = m_f_am_peak_aprmay.loc[m_f_am_peak_aprmay['percent20Secs'] >= m_f_am_peak_aprmay.percent20Secs.quantile(0.8)]

In [ ]:
m_f_am_peak_aprmay.plot()

In [ ]:
m_f_am_peak_aprmay = m_f_am_peak_aprmay['geometry'].to_json()

In [ ]:
mapdawg = folium.Map(location = [36.19, -86.65453], zoom_start = 12)
style1 = {'color':'blue', 'weight':3}
folium.GeoJson(m_f_am_peak_aprmay, style_function=lambda x:style1).add_to(mapdawg)
style2 = {'color':'red', 'weight':2}
folium.GeoJson(route_55_zoe1, style_function=lambda x:style2).add_to(mapdawg)
mapdawg.save('../data/route_55.html')
mapdawg

In [22]:
route_55_zoe1.to_csv('janfeb_pm_peak_out_55.csv')
route_55_zoe2.to_csv('febmar_pm_peak_out_55.csv')
route_55_zoe3.to_csv('marapr_pm_peak_out_55.csv')
route_55_zoe4.to_csv('aprmay_pm_peak_out_55.csv')
route_55_zoe5.to_csv('mayjun_pm_peak_out_55.csv')
route_55_zoe6.to_csv('junjul_pm_peak_out_55.csv')